In [ ]:
import json 
from collections import Counter
from tqdm import tqdm 
from colorama import Style,Fore
import os
from dotenv import load_dotenv
import time
load_dotenv()

In [13]:
from utils1 import openJson,saveJson

## Filtering 1

- videosF1.json contain the videos with fr-FR on the language parameter

In [ ]:
videosR3 = openJson("../jsons/videosR3.json")
len(videosR3)

In [ ]:
videosF1 = []
for video in videosR3:
    if video['langue']== 'fr-FR':
        videosF1.append(video)
saveJson("../jsons/videosF1.json",videosF1)
len(videosF1)

## Filtering 2

### Detecter les videos des Chaines TV 

In [ ]:
chainesTv = [
    "France 2", "France 3", "France 4","France 5","Franceinfo",
    "BFMTV", "C8", "CStar", "Gulli", "Cnews",
    "Canal+", "Planète+", "LCI", "Paris première",
    "6ter", "Arte", "M6", "W9",
    "TFX", "TMC", "NRJ12", "TF1","La Chaîne parlementaire",
    "Chérie 25", "RMC"
]
chainesTv = [nomTV.lower().replace(" ", "") for nomTV in chainesTv]

print(len(chainesTv))
print(chainesTv)

In [ ]:
videosF1 = openJson("../jsons/videosF1.json")
len(videosF1)

In [118]:
channels = openJson("../../collecting/jsons/channels.json")
def getChannelName(channelId):
    for channel in channels:
        if channel['id_chaine']==channelId:
            return channel['nom_chaine'].lower().replace(" ", "")

In [ ]:
chainesTVCounter = Counter()
videosChainesTV = []
for video in videosF1:
    channelName = getChannelName(video['id_chaine'])
    for name in chainesTv:
        if name in channelName:  
            print(name,' ',channelName,' ',video['id_video'])
            chainesTVCounter[name]+=1
            videosChainesTV.append(video['id_video'])

In [ ]:
chainesTVCounter

In [ ]:
import matplotlib.pyplot as plt

labels = list(chainesTVCounter.keys())
sizes = list(chainesTVCounter.values())

plt.figure(figsize=(10, 6))
bars = plt.bar(labels, sizes, color='skyblue')

plt.title("Répartition des vidéos des chaînes TV", fontsize=14)
plt.xlabel("Chaînes TV", fontsize=12)
plt.ylabel("Nombre de vidéos", fontsize=12)
plt.xticks(rotation=45, ha='right')

total_videos_tv = sum(sizes)
plt.text(len(labels) - 2, max(sizes) * 0.8, f"Total : {total_videos_tv} vidéos", fontsize=12, ha='center')
plt.text(len(labels) - 2, max(sizes) * 0.74, "4.62% d'ensemble des vidéos", fontsize=12, ha='center')

# Ajouter les nombres au-dessus des barres et les rendre en gras
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, yval + 1, f'{int(yval)}', ha='center', va='bottom', fontsize=12)

plt.tight_layout()
plt.show()


In [ ]:
(218/len(videosF1))*100

### Creer videosF2.json

In [ ]:
videosChainesTV

In [ ]:
len(videosChainesTV)

In [ ]:
videosF2 = []
for video in videosF1:
    if video['id_video'] not in videosChainesTV:
        videosF2.append(video)
saveJson("../jsons/videosF2.json",videosF2)

verification

In [ ]:
len(videosF1)-len(videosF2)

In [ ]:
len(videosF1)

## Filtering 3

### With Embedding model

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("dangvantuan/sentence-camembert-base")

In [6]:
from sentence_transformers import util

def calculateSimilarity(videoMD, youtubequeries):
    embedding_video = model.encode([videoMD], convert_to_tensor=True, normalize_embeddings=True)
    embeddings_queries = model.encode(youtubequeries, convert_to_tensor=True, normalize_embeddings=True)
    scores = util.cos_sim(embedding_video, embeddings_queries)[0]
    simScore = scores.mean().item()
    return simScore


In [ ]:
requete = "Mes livres préférés pour l'AUTONOMIE/AUTOSUFFISANCE : potager, élevage, cuisine, transfo ect)"

documents =  [
    "autosuffisance potager",
    "autosuffisance poêle à bois bouilleur",
    "autosuffisance maison torchis",
    "autosuffisance aquaponie",
    "autosuffisance filtration naturelle",
    "autosuffisance fumage",
    "autosuffisance legumes",
    "autosuffisance agriculture biologique",
    "autosuffisance arbres fruitiers",
    "autosuffisance mangeoire automatique",
    "autosuffisance sol vivant",
    "autosuffisance compost",
    "autosuffisance décomposeurs",
    "autosuffisance fertilisant",
    "autosuffisance rotation des cultures",
    "autosuffisance salades",
    "autosuffisance plantation",
    "autosuffisance légumineuses",
    "autosuffisance microferme",
    "autosuffisance cultures sous les arbres",
    "autosuffisance pollinisation",
    "autosuffisance plantes",
    "autosuffisance tuteurage",
    "autosuffisance germination",
    "autosuffisance maison en ossature bois",
    "autosuffisance fruitier",
    "autosuffisance fromage",
    "autosuffisance foin",
    "autosuffisance sol",
    "autosuffisance haricots",
    "autosuffisance eau",
    "autosuffisance pomme de terre",
    "autosuffisance itinéraire technique",
    "autosuffisance sécheresse",
    "autosuffisance cendre",
    "autosuffisance changement climatique",
    "autosuffisance bio",
    "autosuffisance lombricompost",
    "autosuffisance courgettes",
    "autosuffisance azote",
    "autosuffisance marc de café",
    "autosuffisance cultures précoces",
    "autosuffisance fertilisation organique",
    "autosuffisance exploitation agricole",
    "autosuffisance poullailler",
    "autosuffisance cultivate",
    "autosuffisance amendement",
    "autosuffisance fientes de poulet",
    "autosuffisance fumier",
    "autosuffisance terres",
    "autosuffisance eau dynamisée",
    "autosuffisance résilience",
    "autosuffisance lombricomposteur",
    "autosuffisance plantes médicinales",
    "autosuffisance chaux",
    "autosuffisance choux Daubenton",
    "autosuffisance vivace",
    "autosuffisance tracteur",
    "autosuffisance panneau solaire",
    "autosuffisance installation solaire",
    "autosuffisance glycol",
    "autonomie potager",
    "autonomie poêle à bois bouilleur",
    "autonomie maison torchis",
    "autonomie aquaponie",
    "autonomie filtration naturelle",
    "autonomie fumage",
    "autonomie legumes",
    "autonomie agriculture biologique",
    "autonomie arbres fruitiers",
    "autonomie mangeoire automatique",
    "autonomie sol vivant",
    "autonomie compost",
    "autonomie décomposeurs",
    "autonomie fertilisant",
    "autonomie rotation des cultures",
    "autonomie salades",
    "autonomie plantation",
    "autonomie légumineuses",
    "autonomie microferme",
    "autonomie cultures sous les arbres",
    "autonomie pollinisation",
    "autonomie plantes",
    "autonomie tuteurage",
    "autonomie germination",
    "autonomie maison en ossature bois",
    "autonomie fruitier",
    "autonomie fromage",
    "autonomie foin",
    "autonomie sol",
    "autonomie haricots",
    "autonomie eau",
    "autonomie pomme de terre",
    "autonomie itinéraire technique",
    "autonomie sécheresse",
    "autonomie cendre",
    "autonomie changement climatique",
    "autonomie bio",
    "autonomie lombricompost",
    "autonomie courgettes",
    "autonomie azote",
    "autonomie marc de café",
    "autonomie cultures précoces",
    "autonomie fertilisation organique",
    "autonomie exploitation agricole",
    "autonomie poullailler",
    "autonomie cultivate",
    "autonomie amendement",
    "autonomie fientes de poulet",
    "autonomie fumier",
    "autonomie terres",
    "autonomie eau dynamisée",
    "autonomie résilience",
    "autonomie lombricomposteur",
    "autonomie plantes médicinales",
    "autonomie chaux",
    "autonomie choux Daubenton",
    "autonomie vivace",
    "autonomie tracteur",
    "autonomie panneau solaire",
    "autonomie installation solaire",
    "autonomie glycol",
    "vie en autarcie potager",
    "vie en autarcie poêle à bois bouilleur",
    "vie en autarcie maison torchis",
    "vie en autarcie aquaponie",
    "vie en autarcie filtration naturelle",
    "vie en autarcie fumage",
    "vie en autarcie legumes",
    "vie en autarcie agriculture biologique",
    "vie en autarcie arbres fruitiers",
    "vie en autarcie mangeoire automatique",
    "vie en autarcie sol vivant",
    "vie en autarcie compost",
    "vie en autarcie décomposeurs",
    "vie en autarcie fertilisant",
    "vie en autarcie rotation des cultures",
    "vie en autarcie salades",
    "vie en autarcie plantation",
    "vie en autarcie légumineuses",
    "vie en autarcie microferme",
    "vie en autarcie cultures sous les arbres",
    "vie en autarcie pollinisation",
    "vie en autarcie plantes",
    "vie en autarcie tuteurage",
    "vie en autarcie germination",
    "vie en autarcie maison en ossature bois",
    "vie en autarcie fruitier",
    "vie en autarcie fromage",
    "vie en autarcie foin",
    "vie en autarcie sol",
    "vie en autarcie haricots",
    "vie en autarcie eau",
    "vie en autarcie pomme de terre",
    "vie en autarcie itinéraire technique",
    "vie en autarcie sécheresse",
    "vie en autarcie cendre",
    "vie en autarcie changement climatique",
    "vie en autarcie bio",
    "vie en autarcie lombricompost",
    "vie en autarcie courgettes",
    "vie en autarcie azote",
    "vie en autarcie marc de café",
    "vie en autarcie cultures précoces",
    "vie en autarcie fertilisation organique",
    "vie en autarcie exploitation agricole",
    "vie en autarcie poullailler",
    "vie en autarcie cultivate",
    "vie en autarcie amendement",
    "vie en autarcie fientes de poulet",
    "vie en autarcie fumier",
    "vie en autarcie terres",
    "vie en autarcie eau dynamisée",
    "vie en autarcie résilience",
    "vie en autarcie lombricomposteur",
    "vie en autarcie plantes médicinales",
    "vie en autarcie chaux",
    "vie en autarcie choux Daubenton",
    "vie en autarcie vivace",
    "vie en autarcie tracteur",
    "vie en autarcie panneau solaire",
    "vie en autarcie installation solaire",
    "vie en autarcie glycol",
    "habitats autonomes potager",
    "habitats autonomes poêle à bois bouilleur",
    "habitats autonomes maison torchis",
    "habitats autonomes aquaponie",
    "habitats autonomes filtration naturelle",
    "habitats autonomes fumage",
    "habitats autonomes legumes",
    "habitats autonomes agriculture biologique",
    "habitats autonomes arbres fruitiers",
    "habitats autonomes mangeoire automatique",
    "habitats autonomes sol vivant",
    "habitats autonomes compost",
    "habitats autonomes décomposeurs",
    "habitats autonomes fertilisant",
    "habitats autonomes rotation des cultures",
    "habitats autonomes salades",
    "habitats autonomes plantation",
    "habitats autonomes légumineuses",
    "habitats autonomes microferme",
    "habitats autonomes cultures sous les arbres",
    "habitats autonomes pollinisation",
    "habitats autonomes plantes",
    "habitats autonomes tuteurage",
    "habitats autonomes germination",
    "habitats autonomes maison en ossature bois",
    "habitats autonomes fruitier",
    "habitats autonomes fromage",
    "habitats autonomes foin",
    "habitats autonomes sol",
    "habitats autonomes haricots",
    "habitats autonomes eau",
    "habitats autonomes pomme de terre",
    "habitats autonomes itinéraire technique",
    "habitats autonomes sécheresse",
    "habitats autonomes cendre",
    "habitats autonomes changement climatique",
    "habitats autonomes bio",
    "habitats autonomes lombricompost",
    "habitats autonomes courgettes",
    "habitats autonomes azote",
    "habitats autonomes marc de café",
    "habitats autonomes cultures précoces",
    "habitats autonomes fertilisation organique",
    "habitats autonomes exploitation agricole",
    "habitats autonomes poullailler",
    "habitats autonomes cultivate",
    "habitats autonomes amendement",
    "habitats autonomes fientes de poulet",
    "habitats autonomes fumier",
    "habitats autonomes terres",
    "habitats autonomes eau dynamisée",
    "habitats autonomes résilience",
    "habitats autonomes lombricomposteur",
    "habitats autonomes plantes médicinales",
    "habitats autonomes chaux",
    "habitats autonomes choux Daubenton",
    "habitats autonomes vivace",
    "habitats autonomes tracteur",
    "habitats autonomes panneau solaire",
    "habitats autonomes installation solaire",
    "habitats autonomes glycol",
    "ferme autonome potager",
    "ferme autonome poêle à bois bouilleur",
    "ferme autonome maison torchis",
    "ferme autonome aquaponie",
    "ferme autonome filtration naturelle",
    "ferme autonome fumage",
    "ferme autonome legumes",
    "ferme autonome agriculture biologique",
    "ferme autonome arbres fruitiers",
    "ferme autonome mangeoire automatique",
    "ferme autonome sol vivant",
    "ferme autonome compost",
    "ferme autonome décomposeurs",
    "ferme autonome fertilisant",
    "ferme autonome rotation des cultures",
    "ferme autonome salades",
    "ferme autonome plantation",
    "ferme autonome légumineuses",
    "ferme autonome microferme",
    "ferme autonome cultures sous les arbres",
    "ferme autonome pollinisation",
    "ferme autonome plantes",
    "ferme autonome tuteurage",
    "ferme autonome germination",
    "ferme autonome maison en ossature bois",
    "ferme autonome fruitier",
    "ferme autonome fromage",
    "ferme autonome foin",
    "ferme autonome sol",
    "ferme autonome haricots",
    "ferme autonome eau",
    "ferme autonome pomme de terre",
    "ferme autonome itinéraire technique",
    "ferme autonome sécheresse",
    "ferme autonome cendre",
    "ferme autonome changement climatique",
    "ferme autonome bio",
    "ferme autonome lombricompost",
    "ferme autonome courgettes",
    "ferme autonome azote",
    "ferme autonome marc de café",
    "ferme autonome cultures précoces",
    "ferme autonome fertilisation organique",
    "ferme autonome exploitation agricole",
    "ferme autonome poullailler",
    "ferme autonome cultivate",
    "ferme autonome amendement",
    "ferme autonome fientes de poulet",
    "ferme autonome fumier",
    "ferme autonome terres",
    "ferme autonome eau dynamisée",
    "ferme autonome résilience",
    "ferme autonome lombricomposteur",
    "ferme autonome plantes médicinales",
    "ferme autonome chaux",
    "ferme autonome choux Daubenton",
    "ferme autonome vivace",
    "ferme autonome tracteur",
    "ferme autonome panneau solaire",
    "ferme autonome installation solaire",
    "ferme autonome glycol",
    "éco-lieu potager",
    "éco-lieu poêle à bois bouilleur",
    "éco-lieu maison torchis",
    "éco-lieu aquaponie",
    "éco-lieu filtration naturelle",
    "éco-lieu fumage",
    "éco-lieu legumes",
    "éco-lieu agriculture biologique",
    "éco-lieu arbres fruitiers",
    "éco-lieu mangeoire automatique",
    "éco-lieu sol vivant",
    "éco-lieu compost",
    "éco-lieu décomposeurs",
    "éco-lieu fertilisant",
    "éco-lieu rotation des cultures",
    "éco-lieu salades",
    "éco-lieu plantation",
    "éco-lieu légumineuses",
    "éco-lieu microferme",
    "éco-lieu cultures sous les arbres",
    "éco-lieu pollinisation",
    "éco-lieu plantes",
    "éco-lieu tuteurage",
    "éco-lieu germination",
    "éco-lieu maison en ossature bois",
    "éco-lieu fruitier",
    "éco-lieu fromage",
    "éco-lieu foin",
    "éco-lieu sol",
    "éco-lieu haricots",
    "éco-lieu eau",
    "éco-lieu pomme de terre",
    "éco-lieu itinéraire technique",
    "éco-lieu sécheresse",
    "éco-lieu cendre",
    "éco-lieu changement climatique",
    "éco-lieu bio",
    "éco-lieu lombricompost",
    "éco-lieu courgettes",
    "éco-lieu azote",
    "éco-lieu marc de café",
    "éco-lieu cultures précoces",
    "éco-lieu fertilisation organique",
    "éco-lieu exploitation agricole",
    "éco-lieu poullailler",
    "éco-lieu cultivate",
    "éco-lieu amendement",
    "éco-lieu fientes de poulet",
    "éco-lieu fumier",
    "éco-lieu terres",
    "éco-lieu eau dynamisée",
    "éco-lieu résilience",
    "éco-lieu lombricomposteur",
    "éco-lieu plantes médicinales",
    "éco-lieu chaux",
    "éco-lieu choux Daubenton",
    "éco-lieu vivace",
    "éco-lieu tracteur",
    "éco-lieu panneau solaire",
    "éco-lieu installation solaire",
    "éco-lieu glycol"
  ]

calculateSimilarity(requete,documents)

In [16]:
contexte_autosuffisance = [
    "Potager familial productif pour l'autonomie en légumes",
    "Élevage de poules, moutons et canards pour l’autonomie alimentaire",
    "Utilisation de plantes médicinales et aromatiques cultivées sur place",
    "Récupération et filtration de l'eau via un puits et osmose inverse",
    "Construction de bacs permacoles faits maison pour une culture efficace",
    "Transformation du sol en terre fertile pour maximiser les rendements",
    "Vie en sobriété heureuse et respect de la nature",
    "Utilisation de semences locales et choix variétal réfléchi",
    "Production locale de viande, œufs et légumes",
    "Utilisation d'une serre pour prolonger les cultures",
    "Métiers artisanaux (ébénisterie, photographie culinaire) intégrés dans le mode de vie",
    "Conception d’un lieu de vie en autonomie énergétique et alimentaire"
]


In [ ]:
videosF2 = openJson("../jsons/videosF2.json")
scores = {}
for video in tqdm(videosF2):
    context = video['titre_video'] + video['description'] + ' '.join(video['tags'])
    score = calculateSimilarity(context,documents)
    print(score,'  ',video['id_video'])
    scores[video['id_video']]=score


In [ ]:
videosF2 = openJson("../jsons/videosF2.json")
scores = {}
for video in tqdm(videosF2):
    context = video['titre_video'] + video['description'] + ' '.join(video['tags'])
    score = calculateSimilarity(context,contexte_autosuffisance)
    print(score,'  ',video['id_video'])
    scores[video['id_video']]=score


### LLM Scoring

In [15]:
from langchain_google_genai import GoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate,PromptTemplate

In [16]:
gemini_flash = GoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=os.getenv("GOOGLE_API_KEY_2"))

In [26]:
sysprompt = """
# La definition d'autossuffinnce

L'autosuffisance est la démarche visant à acquérir la capacité de subvenir par soi-même à ses besoins fondamentaux, 
en premier lieu alimentaires par l'autoconsommation – c'est-à-dire produire, récolter et conserver un maximum de sa propre nourriture, 
souvent en privilégiant le bio, le local et le saisonnier. Plus qu'une simple recherche d'autonomie matérielle, 
elle représente un engagement pour être moins dépendant du système économique et social extérieur, impliquant des choix concrets comme trouver un lieu propice et le concevoir judicieusement (par exemple en permaculture), 
ainsi qu'un changement dans la manière de valoriser son temps et de consommer, formant ainsi le fondement d'une vie plus autonome.

---

# Votre mission

- Décider si une vidéo concerne la thématique de l'autosuffisance (vlogs, interviews, partages d'expériences, tutoriels, conseils...) 
en se basant sur les métadonnées de la vidéo (description, titre, tags).
- Donner un score entier de 1 a 10 pour èvaluer la pertinance de la video à la l'autosuffisance. 

---

# Votre réponse  

- La réponse doit être au format JSON :  
  
    "decision": "oui ou non",
    "justification": "Justifiez votre décision avec des arguments"
    "score": votre èvaluation
  
---

# Exemples des videos sur autosuffisance
  
    "titre_video": "Mes livres préférés pour l'AUTONOMIE/AUTOSUFFISANCE : potager, élevage, cuisine, transfo ect)",
    "description": "00:00 Le guide terre vivante de l'autonomie au jardin \n04:06 Le traité Rustica des arbres fruitiers\n05:08 Jardin-Forêt de Fabrice Desjours\n10:30 La forêt Jardin de Martin crawford\n11:15 Cultiver les champignons de Folko Kullmann\n13:00 Le guide des champignons de France et d'Europe de Guillaume Eyssartier et Paul Roux\n15:08 La bible du jardinage Maison Rustique \n19:07 Faites pousser et dégustez vos protéines de François Couplan et Xavier Mathias\n20:25 Manger sauvage de Richard Mabey\n21:49 Le traité Rustica de la conservation \n25:21 Le traité Rustica de la bonne cuisine maison \n30:00 L'art de la fermentation de Luna Kyung et Camille Oger\n31:40 Prodigieuse lactofermentation de Pascal Labbé\n32:06 L'art de faire son fromage de David Asher \n34:15 Le traité Rustica de la ferme familiale \n37:56 Vivre en autosuffisance \n39:29 L'autosuffisance de John Seymour\n41:40 Petit manuel à l'usage de ceux qui vivent retiré du monde chez Larousse\n42:35 La cuisine de la ferme de Sarah Mayor \n43:40 Le petit traité Rustica de l'apiculteur débutant \n43:58 Les plantes mellifères mois par mois de Jacques Piquée\n44:17 Planter des arbres pour les abeilles de Yves Darricau\n45:05 Arbres fourragers de Jerome Goust\n45:45 Planter des haies pour la biodiversité de Bernard Farinelli \n48:00 Manuel de construction en bambou\n\n\n\n\n\n\n\nLe lien du discord pour participer à la recopie des 5 tomes de la maison rustique : https://discord.gg/mNkvh5cr\nLes 2 vidéos dont je parle pour le livre sur le jardin foret  : https://www.youtube.com/watch?v=D6z-4adGWN4   et https://www.youtube.com/watch?v=WqOt7lfYChU\n\nTu as une question à poser en rapport avec la vidéo ? Je t'invite à le faire en priorité dans les commentaires pour qu'elle puisse servir à tous, mais si tu n'oses pas la poser en public, tu peux me contacter par mail  ! \n\n\n\nMon mail : laptiteortie@gmail.com\n\n\nTu peux me retrouver sur le discord réseautonome : https://reseautono.me/​​​​\nTu y retrouveras des catégories avec plein de conseils dans le domaine de l'autonomie, j'y puise énormément d'infos, beaucoup d'entraide !\n\n\nNotre sol et climat (pour t'aider vis à vis des vidéos de jardin) : Nous vivons dans le Limousin, donc un climat continental, plutôt proche du climat parisien, notre terre est acide et sablo-limoneuse. Nos hivers sont doux même si parfois cela peut geler très fort (-10 en général) et nos étés sont en général secs (et le sont de plus en plus)\n\n\nPour avoir des nouvelles quotidiennement, je t'invite à m'ajouter sur Instagram où je partage tous les jours : https://www.instagram.com/laptiteorti...",
    "tags": "",
---
   
    "titre_video": "Autonomie alimentaire.  Être autosuffisant sur petite surface !",
    "description": "Découvrez le Pirate de la Permaculture et son autonomie alimentaire sur toute petite surface. Picro arrive à être autosuffisant sur une surface d'à peine 800m2... de quoi rêver.\n\nAdhérez à cette chaîne pour obtenir des avantages :\nhttps://www.youtube.com/channel/UC9Q8WeyCb3yxySC3P3mGpBw/join\nPour me soutenir, suivez ce lien : https://fr.tipeee.com/le-jardin-d-emerveille\n\nAu sommaire :\n0:00 - Présentations\n0:53 - Quelles productions\n4:24 - Comment calculer son autosuffisance !\n5:00 - Réduire sa dépendance énergétique\n6:20 - Visite du lieu\n8:08 - Surface et organisation de la production\n10:58 - Gestion de l'eau\n11:51 - Jardin forêt ?\n13:24 - Le pirate Picro et sa chaîne YouTube.\n\nLa chaîne YouTube de Picro : https://www.youtube.com/user/piiicro\n\nPour me soutenir, suivez ce lien : https://fr.tipeee.com/le-jardin-d-emerveille\n\nMerci à vous tous les permapotes d'avoir regardé cette vidéo. :)\nCliquez sur ce lien pour vous abonner : \nhttps://www.youtube.com/channel/UC9Q8WeyCb3yxySC3P3mGpBw",
    "tags": [
      "Autonomie alimentaire. Être autosuffisant sur petite surface !",
      "permaculture",
      "plantes",
      "jardin",
      "biodiversité",
      "agroécologie",
      "potager",
      "d'émerveille",
      "des merveilles",
      "Autonomie alimentaire",
      "Produire sa nourriture",
      "comment créer son potager bio",
      "comment démarrer son potager",
      "comment démarrer un potager",
      "comment faire un potager bio",
      "comment préparer son potager",
      "créer son jardin",
      "créer son potager",
      "faire un potager",
      "etre autosuffisant",
      "Autosuffisant sur petite surface"
    ],
---
    "titre_video": "Lancement et présentation de la chaine : Le jardin de Julien: Le potager autonome ou autosuffisant 🌱",
    "description": "Vidéo de présentation de ma chaine dédiée au potager autosuffisant.\nje pose les bases de fonctionnement de mon potager. Celui ci me permet d' être autonome et autosuffisant a 95% sur les légumes et 60 % sur les fruits. \nJe suis aux antipodes des chaines et techniques du moment ( Permaculture,  non travail du sol, MSV ) difficilement compatibles à mon sens avec les enjeux d'une autosuffisance et ce avec un temps passé restreint au jardin.\nCela ne m'empêche pas de piocher dans certaines techniques très utiles au potager comme celles de jean martin Fortier.\nDe même j' essai de limiter au maximum les investissements, en utilisant du matériel ancien ou de récup pour ne pas exploser le budget.\nBeaucoup de chaines ont de belles idées, et de superbes potagers, mais pas à la portée de toutes les bourses au vu des investissements.\n\nbon visionnage \n\nMa page FB : https://www.facebook.com/profile.php?id=100071363591677",
    "tags": [
      "Potager",
      "potager dolivier",
      "autonome",
      "autosuffisant alimentaire",
      "devenir autosuffisant",
      "potager garden",
      "le jardin de Julien",
      "MSV",
      "permaculture",
      "jean martin fortier",
      "jean martin fortier farm",
      "non travail du sol",
      "légumes",
      "fruits",
      "Antoine le potagiste",
      "loic le jardineur",
      "le jardin d'émerveille",
      "graines",
      "aromatiques"
    ],
"""

userprompt = """
Titre
---
{titre}
Description
---
{descriptions}
tags
---
{tags}
"""

prompt = ChatPromptTemplate([
    ("system", sysprompt),
    ("user", userprompt)
])

chain = prompt | gemini_flash


- Test

In [ ]:
input = {"titre":"La diversification des rotations pour une plus grande autonomie protéique.",
         "descriptions":"[#OnEssaiePourVous] – Episode 1/4. Connaissez-vous la station expérimentale de #Kerguéhennec ? Nous y menons de nombreux essais sur les grandes cultures. Caroline Cocoual, Chargée d’études, vous présente les essais qu’elle mène sur la diversification des rotations pour une plus grande autonomie protéique.",
         "tags":""}
print(chain.invoke(input))

In [28]:
videosF2 = openJson("../jsons/videosF2.json")
reponses = {}
count = 0
temp = 0

for video in tqdm(videosF2):
    input = {"titre":video['titre_video'],
             "descriptions":video['description'],
             "tags":video['tags'],}
    try:
        answer = chain.invoke(input)
        #print(video['id_video'],'/n',answer)
        reponses[video['id_video']]=answer
        count+=1
        temp+=1
        
        if count >= 13:
            print("sleep for 1 min")
            time.sleep(60)
            count=0
        if temp >= 100:
            saveJson("../jsons/llmEvaluation.json",reponses)
            temp=0
    except Exception as e:
        print(f"error {e}")

error 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 41
}
]


 35%|███▌      | 1579/4504 [2:19:30<1:42:59,  2.11s/it]Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 39
}
].


error 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 39
}
]


 35%|███▌      | 1580/4504 [2:19:32<1:43:09,  2.12s/it]Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 37
}
].


error 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 37
}
]


 35%|███▌      | 1580/4504 [2:19:32<4:18:14,  5.30s/it]


KeyboardInterrupt: 